In [1]:
!cat container/Dockerfile

# Build an image that can do training and inference in SageMaker
# This is a Python 3 image that uses the nginx, gunicorn, flask stack
# for serving inferences in a stable way.

FROM ubuntu:18.04

MAINTAINER Amazon AI <sage-learner@amazon.com>


RUN apt-get -y update && apt-get install -y --no-install-recommends \
         wget \
         python3-pip \
         python3-setuptools \
         nginx \
         ca-certificates \
    && rm -rf /var/lib/apt/lists/*

RUN ln -s /usr/bin/python3 /usr/bin/python
RUN ln -s /usr/bin/pip3 /usr/bin/pip

# Here we get all python packages.
# There's substantial overlap between scipy and numpy that we eliminate by
# linking them together. Likewise, pip leaves the install caches populated which uses
# a significant amount of space. These optimizations save a fair amount of space in the
# image, which reduces start up time.
RUN pip --no-cache-dir install numpy==1.16.2 scipy==1.2.1 scikit-learn==0.20.2 pandas flask gunicorn

# Set some environment variabl


<br><Br>
## 컨테이너 빌드 및 등록

In [4]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-kmk

cd container

chmod +x simpleESN/train
chmod +x simpleESN/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to ap-northeast-2 if none defined)
region=$(aws configure get region)
region=${region:-ap-northeast-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"
echo $fullname

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

197354151997.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-kmk:latest
Login Succeeded
Sending build context to Docker daemon  115.7kB
Step 1/11 : FROM ubuntu:18.04
 ---> 476c64862aa8
Step 2/11 : MAINTAINER Amazon AI <sage-learner@amazon.com>
 ---> Using cache
 ---> 64d2a5d395b8
Step 3/11 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          python3-pip          python3-setuptools          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 3883e87b2db1
Step 4/11 : RUN ln -s /usr/bin/python3 /usr/bin/python
 ---> Using cache
 ---> b3ddc1bf1ddb
Step 5/11 : RUN ln -s /usr/bin/pip3 /usr/bin/pip
 ---> Using cache
 ---> 8366a297860a
Step 6/11 : RUN pip --no-cache-dir install numpy==1.16.2 scipy==1.2.1 scikit-learn==0.20.2 pandas flask gunicorn
 ---> Using cache
 ---> 3f21a131ce09
Step 7/11 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> aed86c8cd4da
Step 8/11 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



<br><br>

# Amazon SageMaker에서 커스텀 알고리즘으로 학습과 추론 실행

### 환경 셋업
- SageMaker에서 사용할 S3 버킷을 설정

In [5]:
# S3 prefix
prefix = 'ESN'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

### 세션 생성


In [6]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

### 하이퍼파라미터 정의


In [7]:
hyperparameters = {
        "trainLen" : 2000,    
        "testLen": 2000,
        "initLen": 100
}

### 학습용 데이터 업로드

In [8]:
WORK_DIRECTORY = 'data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

<br><br>

## Estimator 생성 및 학습 실행

- The container name - 이전 쉘스크립트에서 생성한 이름
- The role - 학습을 실행할 IAM 역할(role)
- The instance count - 학습에 사용할 머신의 개수
- The instance type - 학습에 사용할 머신의 인스턴스 타입
- The output path - 학습결과로 생성되는 모델 아티펙트의 저장 위치
- The session - 이전 단계에서 지정한 SageMaker 세션


In [9]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name

In [ ]:
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-kmk:latest'.format(account, region)

esn = sage.estimator.Estimator(image,
                       role, 1, 'ml.c4.2xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       hyperparameters=hyperparameters,    #하이퍼파라미터 추가
                       sagemaker_session=sess)

esn.fit(data_location)

2022-09-06 16:29:04 Starting - Starting the training job...
2022-09-06 16:29:22 Starting - Preparing the instances for trainingProfilerReport-1662481744: InProgress
.........
2022-09-06 16:30:58 Downloading - Downloading input data
2022-09-06 16:30:58 Training - Training image download completed. Training in progress..Starting the training.
['/opt/ml/input/data/training/.ipynb_checkpoints', '/opt/ml/input/data/training/MackeyGlass_t17.csv', '/opt/ml/input/data/training/iris.csv']
Computing spectral radius...
done.
MSE = 1.0228880758912786e-06
Training complete.
